<a href="https://colab.research.google.com/github/Sindhusridasari/minorproject_team5_genomesignalprediction/blob/main/Genome_signal_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive 
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Libraries used
import tensorflow as tf
import os
import keras
import numpy as np
import pandas as pd
from google.colab import files
from itertools import product
from keras.models import load_model
from keras.models import Model
from keras import regularizers
from keras.layers import Dense, Dropout, Activation, Flatten, Layer, merge, Input, Concatenate, Reshape, concatenate,Lambda,multiply,Permute,RepeatVector
from keras.layers.convolutional import Conv1D
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.preprocessing import scale
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.layers import MultiHeadAttention,Embedding
from keras.layers import LeakyReLU
from xgboost import XGBClassifier

In [8]:
#benchmark dataset
# Dataset consising of 1484 enhancer and 1484 non-enhancer
sequences = pd.read_csv('/content/drive/My Drive/enhancers3-i.csv') #enhancer

s = []
for index,row in sequences.iterrows():
	sequence = (row['Sequence'])
	s.append(sequence.upper())

sequences = pd.read_csv('/content/drive/My Drive/non-enhancers3-i.csv') # non-enhancer

for index,row in sequences.iterrows():
	sequence = (row['Sequence'])
	s.append(sequence.upper())
main_seqs = np.copy(s)

print(len(main_seqs))

2968


In [9]:
# Benchmark dataset consisting of strong and weak ( only used when predicting strong and weak)
sequences = pd.read_csv('/content/drive/My Drive/enhancers3-i.csv')

s = []
for index,row in sequences.iterrows():
	sequence = (row['Sequence'])
	s.append(sequence.upper())
print(len(s))

1484


In [10]:
# Below feature extraction and model training is shown for benchmark dataset for enhancer and non-enhancer prediction
# three feature extraction strategy is used
# Same has to be done for strong and weak

In [11]:
# Position based features - same as iPro2L-PSTKNC with some modification (mentioned in comments)

def pstknc_3merp(a,r):
  data= np.copy(s)
  datap=data[0:742]     # This changes based on number of positive samples present in the dataset
  datan=data[742:1484]  # This changes based on number of negative samples present in the dataset 
                          # label for 1484 [0:742] [742:1484]in case of strong and weak
  
  lookup_table = []
  for p in product('ATGC',repeat=a):
    w=''.join(p)
    lookup_table.append(w)

  fp=np.zeros((r,len(datap[0])-(a-1)))
  fn=np.zeros((r,len(datap[0])-(a-1)))
  for i in range(len(datap)):
    for j in range(len(datap[0])-(a-1)):
      q=datap[i][j:j+a]
      t=lookup_table.index(q)
      fp[t,j]=fp[t,j]+1;


  p=1/r                 # Background probability (modified part of the code)

  fp=np.divide(fp,p)
  flog=np.log(fp)
  for i in range(len(flog)):
    for j in range(len(flog[0])):
      if flog[i,j]==float('-inf'):
        flog[i,j]=-10
      if flog[i,j]==float('inf'):
        flog[i,j]= 10

  for i in range(len(datan)):
    for j in range(len(datan[0])-(a-1)):
      q=datan[i][j:j+a]
      t=lookup_table.index(q)
      fn[t,j]=fn[t,j]+1;

  fn=np.divide(fn,p)
  fnlog=np.log(fn)
  for i in range(len(fnlog)):
    for j in range(len(fnlog[0])):
    #for j in range(len(datan[0])-3):
      if fnlog[i,j]==float('-inf'):
        fnlog[i,j]=-10
      if fnlog[i,j]==float('inf'):
        fnlog[i,j]= 10
  
  #z=np.multiply(fp,flog)-np.multiply(fn,fnlog)      # earlier this was used
  z=flog-fnlog                                       # Modified part
  
  d=np.zeros((len(data),len(data[0])-(a-1)))
  for i in range(len(data)):
    for j in range(len(data[0])-(a-1)):
      q=data[i][j:j+a]
      t=lookup_table.index(q)
      d[i,j]=z[t,j]
  
  return d

In [12]:
# Second feature extraction strategy taken from iLearn (haven't read in detail)

def CKSNAP(fastas, gap):
    if gap < 0:
        print('Error: the gap should be equal or greater than zero' + '\n\n')
        return 0

    
    AA = 'ACGT'
    encodings = []
    aaPairs = []
    for aa1 in AA:
        for aa2 in AA:
            aaPairs.append(aa1 + aa2)

    

    for sequence in fastas:
        
        code = []
        for g in range(gap + 1):
            myDict = {}
            for pair in aaPairs:
                myDict[pair] = 0
            sum = 0
            for index1 in range(len(sequence)):
                index2 = index1 + g + 1
                if index1 < len(sequence) and index2 < len(sequence) and sequence[index1] in AA and sequence[
                    index2] in AA:
                    myDict[sequence[index1] + sequence[index2]] = myDict[sequence[index1] + sequence[index2]] + 1
                    sum = sum + 1
            for pair in aaPairs:
                code.append(myDict[pair] / sum)
        encodings.append(code)
    return encodings

In [13]:
# Third feature extraction strategy taken from iLearn (haven't read in detail)

from collections import Counter
def ENAC(fastas, window=5):

    AA = 'ACGT'
    encodings = []
    header = ['#', 'label']
    for w in range(1, len(fastas[0][1]) - window + 2):
        for aa in AA:
            header.append('SW.' + str(w) + '.' + aa)
    #encodings.append(header)

    for sequence in fastas:
        #name, sequence, label = i[0], i[1], i[2]
        code = []
        for j in range(len(sequence)):
            if j < len(sequence) and j + window <= len(sequence):
                count = Counter(sequence[j:j + window])
                for key in count:
                    count[key] = count[key] / len(sequence[j:j + window])
                for aa in AA:
                    code.append(count[aa])
        encodings.append(code)
    return encodings

In [14]:
# Feature ENAC is called here and converted to numpy array
enac=ENAC(s)  # s is the list containing dataset (Enhancer and non-enhancer)
enac=np.copy(enac)
enac.shape

(1484, 784)

In [15]:
# Feature CKSNAP is called here and converted to numpy array
ncp5=CKSNAP(s,5)
ncp5=np.copy(ncp5)
ncp5.shape

(1484, 96)

In [16]:
# Position- based feature is called here for trimer
main3=pstknc_3merp(3,64)   # 3 is for timer, 64 is dimension
main3.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log


(1484, 198)

In [17]:
# concatenated all the three features
# main_seqs shape for benchmark dataset(enhancer, non-enhancer) is (2968,1078)  
#where 2968 is no. of training samples and 1078 is dimension after concatenating all three features

main_seqs=np.concatenate((main3,ncp5,enac),axis=1)

In [18]:
main_seqs.shape

(1484, 1078)

In [19]:
# ANN used as the prediction model with 1078 input nodes, 2 hidden nodes, 2 output nodes

def create_model():
  input1 = Input(shape=(1078,))
  
  net_output=Dense(units=712,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.l2(1e-4),
    activity_regularizer=regularizers.l2(1e-5))(input1)
  #net_output=Activation('relu')(net_output)
  net_output = Dropout(0.5)(net_output)
  #net_output= Flatten()(net_output)
  
  
  net_output=Dense(2)(net_output)
  net_output=Activation('softmax')(net_output)

  return Model(inputs=[input1],outputs=net_output)

In [20]:
# Model evaluating using 10 fold and ANN (Artificial neural network on benchmark dataset)

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix,matthews_corrcoef

cv = KFold(n_splits=10,random_state=42,shuffle=True)


# Label for all 2968 number of samples # label for 1484 in case of strong and weak
y1=np.ones((742,),dtype=np.int16)
y2=np.zeros((742,),dtype=np.int16)
y=np.concatenate((y1,y2),axis=0)


#feature extraction and model fit
acc=[]
se=[]
sp=[]
mcc=[]
j=1;

for train_index, test_index in cv.split(main_seqs):

  X_train, X_test, y_train, y_test = main_seqs[train_index], main_seqs[test_index], y[train_index], y[test_index]
  
  
  #converting to one-hot encoder y_train
  encoder = LabelEncoder()
  encoder.fit(y_train)
  encoded_Y = encoder.transform(y_train)
  y_train = np_utils.to_categorical(encoded_Y)
  
 
  
  model=create_model()

  model.compile(loss='binary_crossentropy',optimizer='adam')
  K.set_value(model.optimizer.learning_rate, 0.001)
  
  checkpointer = ModelCheckpoint(filepath="enhancer_model"+str(j)+".h5", verbose=1, save_best_only=True)
  earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

  #converting to one-hot encoder y_test1
  y_test1=y_test
  encoder = LabelEncoder()
  encoder.fit(y_test1)
  encoded_Y = encoder.transform(y_test1)
  y_test1 = np_utils.to_categorical(encoded_Y)
  
  
  #print(X_train.shape)
  model.fit([X_train], y_train,epochs=100,shuffle=True,validation_data=([X_test],y_test1),callbacks=[checkpointer,earlystopper])

  #model prediction

  # LOAD BEST MODEL to evaluate the performance of the model
  model.load_weights("enhancer_model"+str(j)+".h5")
  
  y_pred = np.argmax(model.predict([X_test]),axis=1)
  tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
  #print(y_pred1)
  #print(y_pred)
  
  print(j)
  j=j+1;
 
  #model evaluation
  accuracy=(tp+tn)/(tp+ fn + tn + fp)
  sensitivity=(tp)/(tp + fn)
  specificity=(tn)/(tn+ fp)

  num=(tp * tn)-(fp * fn )
  den=(tp + fn )*(tn + fp)*(tp + fp )*(tn + fn )
  den=np.sqrt(den)
  mca=num/den

  acc.append(accuracy)
  se.append(sensitivity)
  sp.append(specificity)
  mcc.append(mca)

Epoch 1/100
41/42 [============================>.] - ETA: 0s - loss: 0.5750
Epoch 1: val_loss improved from inf to 0.38921, saving model to enhancer_model1.h5
42/42 [==============================] - 3s 21ms/step - loss: 0.5710 - val_loss: 0.3892
Epoch 2/100
40/42 [===========================>..] - ETA: 0s - loss: 0.3413
Epoch 2: val_loss improved from 0.38921 to 0.34045, saving model to enhancer_model1.h5
42/42 [==============================] - 1s 17ms/step - loss: 0.3408 - val_loss: 0.3405
Epoch 3/100
39/42 [==========================>...] - ETA: 0s - loss: 0.3120
Epoch 3: val_loss improved from 0.34045 to 0.33100, saving model to enhancer_model1.h5
42/42 [==============================] - 1s 17ms/step - loss: 0.3107 - val_loss: 0.3310
Epoch 4/100
42/42 [==============================] - ETA: 0s - loss: 0.2891
Epoch 4: val_loss improved from 0.33100 to 0.30799, saving model to enhancer_model1.h5
42/42 [==============================] - 1s 18ms/step - loss: 0.2891 - val_loss: 0.3080


In [21]:
# Results on bechmark dataset for enhancer and non-enhancer prediction (with 3 features and ANN )

print((np.sum(acc))/10)  # Accuracy
print((np.sum(se))/10)   # Sensitivity
print((np.sum(sp))/10)   # Specificity 
print((np.sum(mcc))/10)  # MCC

0.9898966080174134
0.9893459857846046
0.9900727928117634
0.9796345989863108


In [22]:
# Above all same method is applied for strong and weak enhancer prediction
# Results on bechmark dataset for strong and weak enhancer (with 3 features and ANN )

print((np.sum(acc))/10)    # Accuracy
print((np.sum(se))/10)     # Sensitivity  
print((np.sum(sp))/10)     # Specificity  
print((np.sum(mcc))/10)    # MCC

0.9898966080174134
0.9893459857846046
0.9900727928117634
0.9796345989863108
